In [51]:
!pip install mesa
import mesa
import numpy as np
from IPython.display import Javascript
import copy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
def randomStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

def randomOwnerStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = -1
    # This will break if we add a draw pile and an agent knows the random_card is in the draw pile
    while other_agent == -1:
        other_agent = np.random.choice(random_card.potential_owners)
    return random_card, other_agent

In [65]:
class Disjunction:
    def __init__(self, other_agent, cards):
        self.other_agent = other_agent
        self.cards = cards

    def considerFact(self, fact):
        # If the fact makes one element of the disjunct impossible, delete that element from cards
        # If the fact satisfies one element from the disjunct, delete the disjunct (see collapseExclusiveOrs)
        pass

    def __eq__(self, other):
        if not isinstance(other, Disjunction):
            return False

        return self.other_agent == other.other_agent and self.cards == other.cards

    def __repr__(self):
        return "disjunct_" + str(self.other_agent) + "_" + str(self.cards)

class ExclusiveOr:
    def __init__(self, other_agents, card, quartet_group):
        self.other_agents = other_agents
        self.card = card
        self.quartet_group = quartet_group

    # Returns whether this knowledge is still relevant
    def considerFact(self, fact):
        if self.card != fact.card:
            return True
        if fact.owns:
            # delete the ExclusiveOr
            return False
        else:
            try:
                self.other_agents.remove(fact.agent)
            except ValueError:
                pass
        return True

    def __eq__(self, other):
        if not isinstance(other, ExclusiveOr):
            return False

        return self.other_agents == other.other_agents and self.card == other.card

    def __repr__(self):
        return "exclusiveor_" + str(self.other_agents) + "_" + str(self.card)


class Fact:
    def __init__(self, agent, card, owns = True):
        self.agent = agent
        self.card = card
        self.owns = owns

    def considerFact(self, fact):
        if self.card != fact.card:
            return True
        if self.agent == fact.agent:
            # New fact is either identical to me or an update. So, I am not relevant
            return False
        if self.owns and fact.owns:
            # The card has a new owner; I am irrelivant
            return False
        # At this point, the new fact cannot be contradictory to me, so I stay relevant
        return True

    def getDerivableFacts(self, other_agents):
        derivable_facts = []
        if not self.owns:
            return derivable_facts
        for other_agent in other_agents:
            derivable_facts.append(Fact(other_agent, self.card, owns = False))
        return derivable_facts

    def __eq__(self, other):
        if not isinstance(other, Fact):
            return False

        return self.agent == other.agent and self.card == other.card and self.owns == other.owns

    def __repr__(self):
        return "fact_" + str(self.agent) + "_" + str(self.card) + "_" + str(self.owns)

In [68]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class Hand:
    def __init__(self, model, agent, other_agents):
        self.model = model
        self.agent = agent
        self.other_agents = other_agents
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names, agent, other_agents)
                              for quartet_set in self.model.quartet_sets]

    def getCards(self, flat = False):
        cards = [quarted_group.getCards() for quarted_group in self.quartet_groups]
        if flat:
            cards = flatten(cards)
        return cards

    def addCard(self, new_card):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card.group_name:
                quartet_group.addCard(new_card)
                return quartet_group.isComplete()

    def missingCards(self, flat = False, exclude_unowned_groups = False):
        missing_cards = [group.missingCards() for group in self.quartet_groups]
        if exclude_unowned_groups:
            new_missing_cards = []
            for missing_cards_group in missing_cards:
                if len(missing_cards_group) != 4:
                    new_missing_cards.append(missing_cards_group)
            missing_cards = new_missing_cards

        if flat:
            missing_cards = flatten(missing_cards)
        return missing_cards

    def getGroup(self, group_name):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == group_name:
                return quartet_group

    def getCard(self, asked_card):
        quartet_group = self.getGroup(asked_card.group_name)
        for card in quartet_group.cards:
            if card.card_name == asked_card.card_name:
                return card

    def giveCard(self, asked_card):
        card = self.getCard(asked_card)
        if card.owned:
            card.owned = False
            return True
        return False

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        quartet_group = self.getGroup(group_name)
        quartet_group.updateAgentHasGroup(other_agent_id, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentDoesNotHaveCard(agent_id, card)

class QuartetGroup:
    def __init__(self, group_name, card_names, agent = -1, other_agents = []):
        self.group_name = group_name
        self.card_names = card_names
        self.agent = agent
        self.other_agents = other_agents
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]
        self.knowledge = []
        for card in self.cards:
            card_knowledge = ExclusiveOr(other_agents, card, self)
            self.knowledge.append(card_knowledge)
            # self.potential_owners.append(-1)

    def getCardObject(self, asked_card):
        for card in self.cards:
            if asked_card == card:
                return card

    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
                new_fact = Fact(self.agent, card)
                self.addFact(new_fact)

    def missingCards(self):
        current_cards = self.getCards()
        return [x for x in self.cards if x not in current_cards]

    def getCards(self):
        return [x for x in self.cards if x.owned]

    def isComplete(self):
        return np.all([x.owned for x in self.cards])

    def addFact(self, fact):
        if fact in self.knowledge:
            return
        updated_knowledge = copy.deepcopy(self.knowledge)
        # Update current knowledge base
        for knowledge in self.knowledge:
            keep_knowledge = knowledge.considerFact(fact)
            if not keep_knowledge:
                updated_knowledge.remove(knowledge)

        # Add the fact
        updated_knowledge.append(fact)
        self.knowledge = updated_knowledge
        self.collapseExclusiveOrs()

        # If we learned that an agent now owns a certain card, also add the facts that the other agents do not own the card
        derivable_facts = fact.getDerivableFacts(self.other_agents)
        for derivable_fact in derivable_facts:
            self.addFact(derivable_fact)


    def collapseExclusiveOrs(self):
        collapsing_ors = []
        for knowledge in self.knowledge:
            if type(knowledge) == ExclusiveOr:
                if (len(knowledge.other_agents)) == 1:
                    collapsing_ors.append(copy.copy(knowledge))
        new_facts = []
        for collapsing_or in collapsing_ors:
            new_facts.append(Fact(collapsing_or.other_agents[0], collapsing_or.card, owns = True))
            self.knowledge.remove(collapsing_or)
        for fact in new_facts:
            self.addFact(fact)


    # Get all the cards that a given agent might have
    def getPotentialCardsOfOtherAgent(self, other_agent, excluded_card):
        potential_cards = []
        for card in self.cards:
            if card.owned: continue
            if card == excluded_card: continue
            if other_agent in card.potential_owners:
                potential_cards.append(card)
        return potential_cards

    def updateAgentHasGroup(self, other_agent, excluded_card):
        # Get the potential cards the agent can have
        potential_cards = self.getPotentialCardsOfOtherAgent(other_agent, excluded_card)
        # print("As agent", self.agent, "I know agent", other_agent, "can have cards", potential_cards)
        if len(potential_cards) == 1:
            potential_cards[0].potential_owners = [other_agent]
        else:
            self.knowledge_per_agent[other_agent] = potential_cards
            print("As agent", self.agent, "I have this knowledge:", self.knowledge_per_agent)

    def updateAgentHasCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        card_object.potential_owners = [agent_id]

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        try:
            card_object.potential_owners.remove(agent_id)
        except ValueError:
            pass

    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned

    def __eq__(self, other):
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name

    def __repr__(self):
        return self.group_name + "_" + self.card_name

    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):

    def __init__(self, unique_id, model, strategy):
        super().__init__(unique_id, model)
        self.other_agents = [i for i in range(self.model.num_agents) if i != self.unique_id]
        self.hand = Hand(model, unique_id, self.other_agents)
        self.score = 0
        self.strategy = strategy


    def step(self):
        if not self.model.running:
            return
        # print("I am agent", self.unique_id)
        # print("I currently have these cards:")
        # print(self.hand.getCards())
        # print("I currently do not have these cards:")
        # print(self.hand.missingCards())

        missing_cards = self.hand.missingCards(flat = True, exclude_unowned_groups = True)
        if len(missing_cards) == 0:
            self.model.skipTurn(self.unique_id)
            return

        asked_card, other_agent = self.strategy(self.hand, self.unique_id, self.other_agents)

        got_card = self.model.askForCard(self, other_agent, asked_card)
        if got_card:
            self.addCard(asked_card)
            # print("Now, I have these cards:")
            # print(self.hand.getCards())
            # print("Now I can go again!")
            self.step()

    def addCard(self, new_card):
        has_complete_set = self.hand.addCard(new_card)
        if has_complete_set:
            self.score += 1
            self.model.announceCompleteGroup(self.unique_id, new_card.group_name)

    def giveCard(self, card):
        return self.hand.giveCard(card)

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        self.hand.updateAgentHasGroup(other_agent_id, group_name, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        self.hand.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, asked_agent_id, card):
        self.hand.updateAgentDoesNotHaveCard(asked_agent_id, card)



class QuartetModel(mesa.Model):

    def __init__(self, N, strategies):
        self.running = True
        self.num_agents = N
        self.strategies = strategies
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']),
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']),
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        self.completed_sets = 0
        self.agent_can_ask_owned_card = False
        self.print_announcements = True
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self, self.strategies[i])
            self.schedule.add(a)

        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)

        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)

    def step(self):
        """Advance the model by one step."""
        for agent in self.schedule.agents:
            print(agent.unique_id)
            print(agent.hand.getCards())
            for quartet_group in agent.hand.quartet_groups:
                print(quartet_group.knowledge)
        self.schedule.step()

    def askForCard(self, asking_agent, asked_agent_id, card):
        asking_agent_id = asking_agent.unique_id
        if self.print_announcements:
            print("PA: Agent", asking_agent_id, "asks agent", asked_agent_id, "for card", card)
        asked_agent_object = self.schedule.agents[asked_agent_id]
        self.announceAgentHasGroup(asking_agent, card)
        has_card = asked_agent_object.giveCard(card)
        if has_card:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "had the card and gave it to", asking_agent_id)
            self.announceAgentHasCard(asking_agent_id, card)
        else:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "did not have the card.")
            self.announceAgentDoesNotHaveCard(asked_agent_id, card)
        print("")
        return has_card

    def announceAgentHasGroup(self, asking_agent, card):
        # The asking agent has complete knowledge over its cards, so we can skip this
        # This is always a successful announcement
        group_name = card.group_name
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent]
        excluded_card = None
        if not self.agent_can_ask_owned_card:
            excluded_card = card
        for agent in other_agents:
            agent.updateAgentHasGroup(asking_agent.unique_id, group_name, excluded_card)

    def announceAgentHasCard(self, asking_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent_id]
        for agent in other_agents:
            agent.updateAgentHasCard(asking_agent_id, card)

    def announceAgentDoesNotHaveCard(self, asked_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asked_agent_id]
        for agent in other_agents:
            agent.updateAgentDoesNotHaveCard(asked_agent_id, card)

    def announceCompleteGroup(self, agent, group_name):
        if self.print_announcements:
            print("PA: Agent", agent, "has completed the set", group_name)
        self.completed_sets += 1
        if len(self.quartet_sets) == self.completed_sets:
            self.finishGame()

    def skipTurn(self, agent):
        if self.print_announcements:
            print("PA: Agent", agent, "cannot ask for any card and skips this turn.")

    def finishGame(self):
        print("----------------------------------------------------------------------------")
        print("The game is finished! Here are the scores:")
        for agent in self.schedule.agents:
            print("Agent", agent.unique_id, "has score", agent.score)
        self.running = False

    def getScores(self):
        scores = []
        for agent in self.schedule.agents:
            scores.append(agent.score)
        return scores

In [69]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))

strategyNames = [randomStrategy, randomOwnerStrategy]

strategies = [randomStrategy, randomStrategy, randomStrategy]

total_scores = [0, 0, 0]
for i in range(1):
    model = QuartetModel(3, strategies)
    model.run_model()
    total_scores = np.add(total_scores, model.getScores())

print(total_scores)

<IPython.core.display.Javascript object>

0
[[a_a_1], [b_b_1, b_b_2], [c_c_4]]
[exclusiveor_[1, 2]_a_a_2, exclusiveor_[1, 2]_a_a_3, exclusiveor_[1, 2]_a_a_4, fact_0_a_a_1_True, fact_1_a_a_1_False, fact_2_a_a_1_False]
[exclusiveor_[1, 2]_b_b_3, exclusiveor_[1, 2]_b_b_4, fact_0_b_b_2_True, fact_1_b_b_2_False, fact_2_b_b_2_False, fact_0_b_b_1_True, fact_1_b_b_1_False, fact_2_b_b_1_False]
[exclusiveor_[1, 2]_c_c_1, exclusiveor_[1, 2]_c_c_2, exclusiveor_[1, 2]_c_c_3, fact_0_c_c_4_True, fact_1_c_c_4_False, fact_2_c_c_4_False]
1
[[a_a_4], [], [c_c_1, c_c_2, c_c_3]]
[exclusiveor_[0, 2]_a_a_1, exclusiveor_[0, 2]_a_a_2, exclusiveor_[0, 2]_a_a_3, fact_1_a_a_4_True, fact_0_a_a_4_False, fact_2_a_a_4_False]
[exclusiveor_[0, 2]_b_b_1, exclusiveor_[0, 2]_b_b_2, exclusiveor_[0, 2]_b_b_3, exclusiveor_[0, 2]_b_b_4]
[exclusiveor_[0, 2]_c_c_4, fact_1_c_c_2_True, fact_0_c_c_2_False, fact_2_c_c_2_False, fact_1_c_c_3_True, fact_0_c_c_3_False, fact_2_c_c_3_False, fact_1_c_c_1_True, fact_0_c_c_1_False, fact_2_c_c_1_False]
2
[[a_a_2, a_a_3], [b_b_3, b_b

AttributeError: ignored

In [56]:
print(total_scores)

[1796  687  517]
